In [ ]:
import numpy
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import datasets,svm,metrics
import math
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix

In [ ]:
data_inputs=pd.read_csv('/content/drive/My Drive/DataSet/Sipak_googlenet.csv')
data_outputs=pd.read_csv('/content/drive/My Drive/DataSet/sipak_label_googlenet.csv')
data_inputs=data_inputs.to_numpy()
data_outputs=data_outputs.to_numpy()
data_inputs=data_inputs[:,1:]
data_outputs=data_outputs[:,1]

In [ ]:
print(data_outputs.shape)
print(data_inputs.shape)

In [ ]:
kf=KFold(5,True,random_state=1)

In [ ]:
HMS=50
num_samples = data_inputs.shape[0]
num_feature_elements = data_inputs.shape[1]

In [ ]:
HM_shape=(HMS,num_feature_elements)
HMCR=0.9
PARmin=0.4
PARmax=0.8
bwmin=0.0001
bwmax=1.0
PVB=numpy.empty((num_feature_elements,2))
BW=numpy.zeros((1,num_feature_elements))  
JRmax=1
JRmin=0

In [ ]:
harmony_memory=numpy.random.randint(low=0,high=2,size=HM_shape)

In [ ]:
def reduce_features(solution, features):
    selected_elements_indices = numpy.where(solution ==1)[0]
    reduced_features = features[:, selected_elements_indices]
    return reduced_features

In [ ]:
def classification_accuracy(labels, predictions):
    correct = numpy.where(labels == predictions)[0]
    accuracy = correct.shape[0]/labels.shape[0]
    return accuracy

In [ ]:
def cal_pop_fitness(pop, features, labels, train_indices, test_indices,classifier):
    accuracies = numpy.zeros(pop.shape[0])
    idx = 0

    for curr_solution in pop:
        reduced_features = reduce_features(curr_solution, features)
        train_data = reduced_features[train_indices, :]
        test_data = reduced_features[test_indices, :]

        train_labels = labels[train_indices]
        test_labels = labels[test_indices]
        if classifier=='SVM':
          SV_classifier = sklearn.svm.SVC(kernel='rbf',gamma='scale',C=5000)
          SV_classifier.fit(X=train_data, y=train_labels)

          predictions = SV_classifier.predict(test_data)
          accuracies[idx] = classification_accuracy(test_labels, predictions)
          idx = idx + 1
        if classifier == 'KNN':
          knn=KNeighborsClassifier(n_neighbors=8)
          knn.fit(train_data,train_labels)
          predictions=knn.predict(test_data)
          accuracies[idx]=classification_accuracy(test_labels,predictions)
          idx = idx + 1
        
        if classifier=='MLP':
          classifier = MLPClassifier(max_iter=500,activation = 'relu',solver='adam',random_state=1)
          classifier.fit(train_data,train_labels)
          y_pred = classifier.predict(test_data)
          cm = confusion_matrix(y_pred, test_labels)
          accuracies[idx]=classification_accuracy(test_labels,y_pred)
          idx=idx+1
    return accuracies

In [ ]:
num=math.ceil(HMS/2)
opp_num=HMS-num
classifier='SVM'

In [ ]:
NCHV = numpy.ones((1, num_feature_elements))
best_outputs = []
best_opp_outputs = []
MaxItr = 10
fold=0
for train_indices,test_val_indices in kf.split(data_inputs):
  fold=fold+1
  print("Fold : ",fold)
  val_indices,test_indices=train_test_split(test_val_indices,test_size=0.5,shuffle=True,random_state=6)
  best_test_outputs=[]
  harmony_memory=numpy.random.randint(low=0,high=2,size=HM_shape)
  for currentIteration in range(MaxItr):
      NCHV = numpy.ones((1, num_feature_elements))
      print("Generation : ", currentIteration+1)
        
      JR=JRmax-(currentIteration/MaxItr)*(JRmax-JRmin)
      if numpy.random.rand()<JR:
        opposite_memory=1-harmony_memory
        opp_fitness=cal_pop_fitness(opposite_memory,data_inputs,data_outputs,train_indices,val_indices,classifier)
        fitness=cal_pop_fitness(harmony_memory,data_inputs,data_outputs,train_indices,val_indices,classifier)
        fit_ind = numpy.argpartition(fitness, -num)[-num:]
        opp_fit_ind=numpy.argpartition(opp_fitness,-opp_num)[-opp_num:]

        harmony_memory[num,:]=harmony_memory[fit_ind,:][0]
        harmony_memory[opp_num,:]=opposite_memory[opp_fit_ind,:][0]      
      
      test_fitness=cal_pop_fitness(harmony_memory,data_inputs,data_outputs,train_indices,test_indices,classifier)
      best_test_outputs.append(numpy.max(test_fitness))
      print("Best Test result :",max(best_test_outputs))
      
      fitness = cal_pop_fitness(harmony_memory, data_inputs, data_outputs, train_indices, val_indices,classifier)    
      best_outputs.append(numpy.max(fitness))
      print("Best validation result : ", max(best_outputs))
    
      
      PAR = (PARmax - PARmin) / (MaxItr) * currentIteration + PARmin
      PAR=0.3
      coef = numpy.log(bwmin / bwmax) / MaxItr
      for pp in range(num_feature_elements):
          BW[0, pp] = bwmax * numpy.exp(coef * currentIteration)

      
      for i in range(num_feature_elements):
          ran = numpy.random.rand()
          if ran < HMCR:
              index = numpy.random.randint(0, HMS)
              NCHV[0, i] = harmony_memory[index, i]
              pvbran = numpy.random.rand()
              if pvbran < PAR:
                  pvbran1 = numpy.random.rand()
                  result = NCHV[0, i]
                  if pvbran1 < 0.5:
                      result =1-result

          else:
              NCHV[0, i] = numpy.random.randint(low=0,high=2,size=1)
      
      new_fitness = cal_pop_fitness(NCHV, data_inputs, data_outputs, train_indices, val_indices,classifier)
      if new_fitness > min(fitness):
          min_fit_idx = numpy.where(fitness == min(fitness))
          harmony_memory[min_fit_idx, :] = NCHV
          fitness[min_fit_idx] = new_fitness
      
      
  fitness = cal_pop_fitness(harmony_memory, data_inputs, data_outputs, train_indices, test_indices,classifier)

  best_match_idx = numpy.where(fitness == numpy.max(fitness))[0]
  best_match_idx = best_match_idx[0]

  best_solution = harmony_memory[best_match_idx, :]
  best_solution_indices = numpy.where(best_solution == 1)[0]
  best_solution_num_elements = best_solution_indices.shape[0]
  best_solution_fitness = numpy.max(fitness)

  print("best_match_idx : ", best_match_idx)
  print("best_solution : ", best_solution)
  print("Selected indices : ", best_solution_indices)
  print("Number of selected elements : ", best_solution_num_elements)
  print("Best solution fitness : ", best_solution_fitness)


In [ ]:
data=data_inputs[:,best_solution_indices]

In [ ]:
import pandas as pd
df=pd.DataFrame(data)
df.to_csv('/content/drive/My Drive/Herlev/Herlev_CSV/Googlenet_Selected.csv')